In [ ]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
using LinearAlgebra, Plots
import Convex as cvx 
import MeshCat as mc 
using Test
using Random
import Convex as cvx 
import ForwardDiff as FD
import ECOS  
using Random    
using ControlSystems
#include(joinpath(@__DIR__,"ref_traj.jl"))
include(joinpath(@__DIR__,"planar_hexrotor_dynamics.jl"))

In [ ]:
# Controls equilibrium point
function controls_equilibrium(u0::Vector, X̄, model, ode; tol= 1e-6, max_iters= 100, verbose= false)::Vector{Vector{Float64}}

    "
        Determining the equilibrium point for controls at hover
            - use Newton's method (minimize over u for least squares cost)
            - initial guess= hover location for planar hexarotor ([(9.81*mass/n_inputs)*ones(n_inputs) for i = 1:(N-1)])
    " 

    # initialize U
    U = [zeros(eltype(u0), length(u0)) for i= 1:max_iters] #empty vector for Newton guesses
    U[1]= u0 #initialize with first guess (which is simply counteracting gravity (mg/n_inputs)

    #wind_force= [0.0;0.0;0.0]

    for i = (1:max_iters-1)
        
        residual= (1/2) * (ode(model, X̄, U[i]))'* ode(model, X̄, U[i]) # least squares cost function
        δfδu= FD.jacobian(du -> ode(model, X̄, du), U[i]) # jacobian of f with respect to u

        ∇residual= (δfδu)' * ode(model, X̄, U[i]) # gradient of cost function- to solve for u, this should be zero
        ∇²residual= ((δfδu)' * δfδu) + 1e-4*(I(6)) # hessian (gauss newton version) of cost function 
        # regularization term to ensure full rank

        norm_residual= norm(residual)

        if verbose 
            print("iter: $i    |residual|: $norm_residual   \n")
        end

        # check against tol
        # if converged, return U[1:i]
        if norm_residual < tol
            return U[1:i]
        end

        Δu= ∇²residual\(-∇residual) # Newton step
        #println(Δu)
        # update U
        U[i+1]= U[i] + vec(Δu)
        #println(U[i+1])
    end
    error("Newton's method did not converge")
end

# converting from vector of vectors <-> matrix 
function mat_from_vec(X::Vector{Vector{Float64}})::Matrix
    # convert a vector of vectors to a matrix 
    Xm = hcat(X...)
    return Xm 
end

In [ ]:
# Drone parameters
# Ixx= 2.331e2
# Iyy= 2.322e2
# Izz= 4.022e2
# model = (mass= 6.728,
#     J= Diagonal([Ixx, Iyy, Izz]), 
#     gravity= [0,0,-9.81],
#     L= 0.5375, 
#     kf= 0.1, # constant thrust coefficient 
#     km= 0.01, # propeller drag coefficient
#     dt= 0.01#, # 100 horizon
# )
#### MODEL PARAMETERS ####
Ixx= 2.331e2
Ixy= -0.7923
Ixz= 2.859
Iyx= -.7923
Iyy= 2.322e2
Iyz= 2.187
Izx= 2.859
Izy= 2.187
Izz= 4.022e2
model = (mass= 6.728,
    J= Diagonal([Ixx -Ixy -Ixz; -Iyx Iyy -Iyz; -Izx -Izy Izz]), #inertia matrix- determined from CAD
    gravity= [0,0,-9.81],
    L= 0.5375, 
    kf= 13.0,#13, # constant thrust coefficient- this was guessed based on the quadrotor dynamics (linear relationship based on mass...)
    km= 0.0245,#0.0245, # propeller drag coefficient (0.0245 for quadrotor (0.5kg)
    dt= 0.02, # frequency
    α= 0.49, # radians
    β= 0.33, # radians
    wind_mean= [2.0;0.0;0.0] #[2.0;0.0;0.0], # wind force in x,y,z directions
    #wind_stdev= [0.5;0.5;0.5] #[0.5;0.5;0.5]
)

# states and controls 
nx = 12 
nu = 6

# problem size 
dt = model.dt
tf = 10
t_vec = 0:dt:tf
N = length(t_vec)
m= model.mass

# Generate hover trajectory for hexrotor

# initial condition of zeros
X̄= [0.0, 0.0, 3.0, 0.0, 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]


# initial guess for control equilibrium point- I made this the hover thrust for the planar hexrotor
u0= [(9.81*m/(nu)); (9.81*m/(nu)); (9.81*m/(nu)); (9.81*m/(nu)); (9.81*m/(nu)); (9.81*m/(nu))] 
#u0= [0.0; 10.0; 0.0; 0.0; 0.0; 0.0] 

U= controls_equilibrium(u0, X̄, model, hexrotor_dynamics, tol= 1e-6, max_iters= 100, verbose= true)
Ū= U[end] # control equilibrium point! will linearize around this point
println("U equilibrium point= ", Ū)

# m= model.mass
# U_=[(9.81*m/nu)*ones(nu)]


#K_matrix= []
# instantiate S and K 
# P = [zeros(nx,nx) for i = 1:N]
# K = [zeros(nu,nx) for i = 1:N-1]
# initialize S[N] with Qf 
#P[N] = deepcopy(Qf)

# linearize the discretized dynamics about hover
# state_dynamics(X_)= rk4(model,hexrotor_dynamics, X_, U_, dt)
# control_dynamics(U_)= rk4(model, hexrotor_dynamics, X_, U_, dt)
# A = FD.jacobian(state_dynamics, X_)
# B = FD.jacobian(control_dynamics, U_)


# calculate A and B matrices (jacobians of discrete dynamics with respect to X and U)
A= FD.jacobian(dx -> rk4(model, hexrotor_dynamics, dx, Ū, dt), X̄)
B= FD.jacobian(du -> rk4(model,hexrotor_dynamics, X̄, du, dt), Ū)

@assert norm(X̄- rk4(model, hexrotor_dynamics, X̄, Ū, dt)) < 1e-3
#println(X̄)
#println(rk4(model, hexrotor_dynamics, X̄, Ū, dt))
println("norm between X̄ and discretized dynamics= ", norm(X̄- rk4(model, hexrotor_dynamics, X̄, Ū, dt)))

# # calculate infinite gain, k

# cost terms 
Q = diagm(ones(nx))* 10
# Qf = 10*Q
# R = 0.001*diagm(ones(nu))

# cost terms 
# Q = diagm(ones(nx))*10
# Qf = 10*Q
R = 0.1*diagm(ones(nu))

# #### COST TERMS ####
# Q = diagm([100,100,100,
# 10,10,10,
# 100,100,100,
# 10,10,10])

Qf = 10*Q

#R = 0.01*diagm(ones(nu))
#R = 1.0*diagm(ones(nu))

# for k = N-1: -1:1 
#     K[k]= (R + B'*P[k+1]*B)\ (B'*P[k+1]*A) 
#     P[k]= Q + A'*P[k+1]*(A- B*K[k]) 
# end 
Kinf= B'
max_iters= 1e6
P= deepcopy(Q) #initialize p
for i= 1:max_iters
    Kinf = (R .+ B' * P * B)\  (B' * P * A)
    P_recursion = Q + A' * P * A - A' * P * B * Kinf
    
    if norm(P-P_recursion) <= 1e-6
        break
    end
    # update P for next iteration
    P = 1*P_recursion
end



In [ ]:
######### simulation stuff
Random.seed!(1) 

Δx = [zeros(nx) for i = 1:N]
Δu = [zeros(nu) for i = 1:N-1]

X= [zeros(nx) for i = 1:N]
U= [zeros(nu) for i = 1:N-1]

u_min= -100
u_max= 100


#X[1] = X̄ + 0.001* randn(nx) # add a little bit of noise...
X[1] = [0.5;0.5;2.5;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0] #+ 0.01* randn(nx) # add a little bit of noise...
#simulate this controlled system with rk4(params_real, ...)
for i = 1:N-1
    # calculate control inputs
    # u_lqr = -K[i] * (Δx[i])
    # Δu= u_lqr + U__
    # Δu[i] = -Kinf * (Δx[i]) 
    #Δu= u_lqr + U__
    # simulate the system wit nonlinear dynamics
    U[i]= clamp.(-Kinf * (X[i] - X̄) +Ū, u_min, u_max) #+ 0.01*randn(nu)
    X[i+1]= rk4(model, hexrotor_dynamics,(X[i]), U[i], dt) #+ 0.001*rand(nx)
    #Δx[i+1]= A*(Δx[i]) + B*(Δu[i]) #+ 0.01* randn(nx)

    # check absolutes
    #X[i]= Δx[i] + X_
    #U[i]= Δu[i] + U_
end



In [ ]:
Xm = mat_from_vec(X)
Um = mat_from_vec(U)

display(plot(t_vec,Xm[1:3,:]',title = "Position",
                xlabel = "Time (s)", ylabel = "Position (m)",
                label = ["x" "y" "z"], legend=:right))

display(plot(t_vec[1:end-1],Um',title = "Control Effort",
            xlabel = "Time (s)", ylabel= "Rotor Speed (rad/s)",label = ["Rotor 1" "Rotor 2" "Rotor 3" "Rotor 4" "Rotor 5" "Rotor 6"]) ) 

display(animate_hexrotor(X, dt))

In [ ]:
index= 1
for i= 1:N
    if X[i][1]<= 0.05 && X[i][2] <= 0.05 && X[i][3] >2.95
        return index
    else
        index= index+1  
    end
end
println(index*dt)